# Excercis 3 - Pydantic

## 0. Field validation

### Registrera model 

In [1]:
from pydantic import BaseModel, Field, ValidationError


class Registration(BaseModel):
    username: str = Field(min_length=3)
    password: str = Field(min_length=8)

### Test with valid data

In [2]:
reg = Registration(username="marian", password="supersecret")
print(reg)

username='marian' password='supersecret'


### Test with invalid data

In [3]:
try:
    Registration(username="ab", password="123")
except ValidationError as e:
    print(e)

2 validation errors for Registration
username
  String should have at least 3 characters [type=string_too_short, input_value='ab', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/string_too_short
password
  String should have at least 8 characters [type=string_too_short, input_value='123', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/string_too_short


## 1. User data

### a) User BaseModel (id+name)

In [4]:
from pydantic import BaseModel, ValidationError


class User(BaseModel):
    id: int
    name: str

### Valid instance

In [5]:
user_ok = User(id=1, name="Marian")
print(user_ok)

id=1 name='Marian'


Invalid instance (id as string)

In [6]:
try:
    User(id="not_an_int", name="Marian")
except ValidationError as e:
    print(e)

1 validation error for User
id
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='not_an_int', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/int_parsing


## b) Person BaseModel with validation
### Fields: name, age, email, favourite_pet

In [7]:
from pydantic import BaseModel, Field, EmailStr


class Person(BaseModel):
    name: str = Field(min_length=2)
    age: int = Field(ge=0, le=130)
    email: EmailStr
    favourite_pet: str = Field(min_length=2)

### Test valid 

In [8]:
p_ok = Person(name="Marian", age=25, email="marian@example.com", favourite_pet="dog")
print(p_ok)

name='Marian' age=25 email='marian@example.com' favourite_pet='dog'


### Test invalid

In [9]:
from pydantic import ValidationError

tests = [
    {
        "name": "M",
        "age": 25,
        "email": "marian@example.com",
        "favourite_pet": "dog",
    },  # name too short
    {
        "name": "Marian",
        "age": -1,
        "email": "marian@example.com",
        "favourite_pet": "dog",
    },  # age invalid
    {
        "name": "Marian",
        "age": 25,
        "email": "not_an_email",
        "favourite_pet": "dog",
    },  # invalid email
]

for t in tests:
    try:
        Person(**t)
    except ValidationError as e:
        print("\n--- ValidationError ---")
        print(e)


--- ValidationError ---
1 validation error for Person
name
  String should have at least 2 characters [type=string_too_short, input_value='M', input_type=str]
    For further information visit https://errors.pydantic.dev/2.12/v/string_too_short

--- ValidationError ---
1 validation error for Person
age
  Input should be greater than or equal to 0 [type=greater_than_equal, input_value=-1, input_type=int]
    For further information visit https://errors.pydantic.dev/2.12/v/greater_than_equal

--- ValidationError ---
1 validation error for Person
email
  value is not a valid email address: An email address must have an @-sign. [type=value_error, input_value='not_an_email', input_type=str]


### c) Replicate with a normal Python class

In [10]:
import re


class PersonPlain:
    def __init__(self, name: str, age: int, email: str, favourite_pet: str):
        if not isinstance(name, str) or len(name) < 2:
            raise ValueError("name must be a string with min length 2")

        if not isinstance(age, int) or not (0 <= age <= 130):
            raise ValueError("age must be an int between 0 and 130")

        # simple email validation
        if not isinstance(email, str) or not re.match(r"^[^@]+@[^@]+\.[^@]+$", email):
            raise ValueError("email must be a valid email string")

        if not isinstance(favourite_pet, str) or len(favourite_pet) < 2:
            raise ValueError("favourite_pet must be a string with min length 2")

        self.name = name
        self.age = age
        self.email = email
        self.favourite_pet = favourite_pet

    def __repr__(self):
        return f"PersonPlain(name={self.name!r}, age={self.age}, email={self.email!r}, favourite_pet={self.favourite_pet!r})"

### Test it

In [11]:
pp = PersonPlain("Marian", 25, "marian@example.com", "dog")
print(pp)

try:
    PersonPlain("M", -1, "bademail", "d")
except ValueError as e:
    print("ValueError:", e)

PersonPlain(name='Marian', age=25, email='marian@example.com', favourite_pet='dog')
ValueError: name must be a string with min length 2


## 2. Validate data from API using Pydantic

### Fetch one joke 

In [12]:
import requests

headers = {"Accept": "application/json"}
response = requests.get("https://icanhazdadjoke.com/", headers=headers)

data = response.json()
data

{'id': 'LRf2obFBskb',
 'joke': 'Don’t interrupt someone working intently on a puzzle. Chances are, you’ll hear some crosswords.',
 'status': 200}

## a) Create Pydantic model (Joke)

In [13]:
from pydantic import BaseModel


class Joke(BaseModel):
    id: str
    joke: str

## b) Validate + access fields

In [14]:
j = Joke.model_validate(data)
print("ID:", j.id)
print("Joke:", j.joke)

ID: LRf2obFBskb
Joke: Don’t interrupt someone working intently on a puzzle. Chances are, you’ll hear some crosswords.


## c) Add computed field

In [15]:
from pydantic import computed_field


class JokeWithStats(BaseModel):
    id: str
    joke: str

    @computed_field
    @property
    def words_in_joke(self) -> int:
        """returns number of words in the joke"""
        return len(self.joke.split())

In [16]:
j2 = JokeWithStats.model_validate(data)
print("Words in joke:", j2.words_in_joke)

Words in joke: 14


## d) Fetch 10 jokes

In [17]:
import time

jokes = []

for i in range(10):
    r = requests.get("https://icanhazdadjoke.com/", headers=headers)
    d = r.json()

    jokes.append(JokeWithStats.model_validate(d))

    print(f"{i + 1}/10 validated")
    time.sleep(5)

1/10 validated
2/10 validated
3/10 validated
4/10 validated
5/10 validated
6/10 validated
7/10 validated
8/10 validated
9/10 validated
10/10 validated


In [18]:
# Show first 2 as proof
jokes[:2]

[JokeWithStats(id='1wkqrcNCljb', joke="Did you know that protons have mass? I didn't even know they were catholic.", words_in_joke=14),
 JokeWithStats(id='HJJY0LR7hyd', joke='Where did you learn to make ice cream? Sunday school.', words_in_joke=10)]